In [7]:
from tensorflow import keras
from CustomLayers import *

In [ ]:
Model_path = "C:/Users/uqcche32/OneDrive - The University of Queensland/PhD/HPC_Results/Sugarcane_disease/ML/Single_Head_Residual_attention_realForm32_16Local"

In [9]:
model_folder = "pachy_MultiHeadAttentionLNN_1"
full_path = Model_path + "/" + model_folder
model = keras.models.load_model(full_path,custom_objects={"MultiHead_QKV_BlockAttention": MultiHead_QKV_BlockAttention})

ValueError: Could not find matching function to call loaded from the SavedModel. Got:
  Positional arguments (1 total):
    * Tensor("x:0", shape=(None, 2608, 32), dtype=float32)
  Keyword arguments: {}

Expected these arguments to match one of the following 1 option(s):

Option 1:
  Positional arguments (1 total):
    * [TensorSpec(shape=(None, 2608, 32), dtype=tf.float32, name='x/0')]
  Keyword arguments: {}

In [ ]:
model.summary()